# Árboles de decisión

Todos hemos visto ejemplos de árboles de decisión:

<img src="ml53.png">

## Partes

<img src="ml54.png">

## ¿Cómo generamos árboles de decisión? 

Los árboles de decisión son algoritmos inductivos. Es decir, aprende patrones directamente de ejemplos. 

Algunos de los algoritmos mas usuales son:

* Hunt, en el cual se basan los siguientes.

* CART

* ID3

* C4.5

* SliQ y SPRINT para tablas muy grandes

## Algoritmo de Hunt

Es un algoritmo inductivo, recursivo y oportunista que busca un mínimo local:

* **Inductivo.** Se genera directamente basado en ejemplos.

* **Recursivo.** Repite el proceso varias veces.

* **Oportunista.** En cada paso toma la decisión mas favorable sin pensar en pasos anteriores o futuros.

La solución encuentra una solución óptima (minimiza el error) localmente. 

Funciona partiendo los datos de entrenamiento en grupos cada vez mas pequeños.

Para un nodo **T** que tiene un conjunto de elementos **D_T**:

- Si todos los elementos de **T** son de la misma clase, dicho nodo es una hoja de esa clase. 
- Si no lo son, se aplica un criterio **S** que parte **D_T** en subgrupos. Para cada salida del criterio de partición se crea un nodo *hijo* de **T** y los elementos de **D_T** se reparten entre estos nodos hijos.
- Repetimos el proceso para cada nodo hijo.

Hay varias maneras de de partir un conjunto de elementos en un nodo. El objetivo general en cada paso es maximizar la **pureza** (una partición es 100% pura si todos los elementos en cada subnodo son de la misma clase).



Vivienda|Estado civil|Salario|Impago
:--:|:--:|:--:|:--:
Sí|Soltero|125|No
No|Casado|100|No
No|Soltero|70|No
Sí|Casado|120|No
No|Divorciado|95|Sí
No|Casado|60|No
Sí|Divorciado|220|No
No|Soltero|85|Sí
No|Casado|75|No
No|Soltero|90|Sí



<img src="ml55.png">
<img src="ml56.png">
<img src="ml57.png">
<img src="ml58.png">

## Criterio de partición

Un criterio de partición de una generación de árboles debe responder a las siguientes preguntas:
* **¿Cómo separamos los elementos en cada paso?**
* **¿Cuándo dejamos de separar elementos?** En teoría podríamos seguir separando hasta que la pureza de cada hoja fuera 1, pero en la práctica esto genera árboles muy complejos y sobreajustados (pues llegaría a memorizar cada observación en una hoja distinta).

Para un nodo en concreto, el algoritmo prueba, para cada variable, todas las combinaciones posibles. Hay muchas maneras de hacer particiones.

<img src="ml59.png">

## ¿Qué partición elegimos?

Elegiremos aquella que maximice la pureza respecto del nodo padre. A este criterio se le llama **ganancia de información (information gain)**. Para calcularla, usamos alguna de las siguientes mediciones de pureza:

$$Entropy(t)=-\sum_{i=0}^{c-1}p(i|t)\log_2p(i|t)$$
$$Gini(t)=1-\sum_{i=0}^{c-1}p(i|t)^2$$
$$Clasification\,error(t)=1-\max_{i}p(i|t)$$

**Ejemplo con entropía**

Calculamos la entropía de la raíz
<img src="ml55.png">
<img src="ml60.png">

Para cada posible nodo hijo, calculamos la entropía de dicho nodo. 

Por ejemplo, probamos primero para **Vivienda en propiedad**:
<img src="ml61.png">

Ahora probamos para **Estado civil**:
<img src="ml62.png">

E incluso podemos tomar estado civil con otra parición:
<img src="ml63.png">

## ¿Cuándo paramos de crear particiones?

<img src="ml64.png">

In [4]:
import numpy as np
import pandas as pd

# Datos de ejemplo
edad = np.array([15, 20, 18, 22, 16])
altura = np.array([150, 180, 165, 175, 160])
juega_futbol = np.array(['Sí', 'No', 'Sí', 'Sí', 'No'])

# Crear un DataFrame para facilitar el manejo de los datos
df = pd.DataFrame({'Edad': edad, 'Altura': altura, 'Juega al Fútbol': juega_futbol})

# Mapear 'Sí' a 1 y 'No' a 0 en la columna 'Juega al Fútbol'
df['Juega al Fútbol'] = df['Juega al Fútbol'].map({'Sí': 1, 'No': 0})
df

,Edad,Altura,Juega al Fútbol
0,15,150,1
1,20,180,0
2,18,165,1
3,22,175,1
4,16,160,0


In [5]:

def calculate_gini_index(feature_values, target_values):
    # Obtener valores únicos de la característica
    unique_values = np.unique(feature_values)
    
    gini_index = float('inf')  # Inicializar con infinito para encontrar el  nos permite difinir que siempre se tomar al minimo
    
    for value in unique_values:
        # Dividir los datos en dos conjuntos
        left_subset = target_values[feature_values <= value]
        right_subset = target_values[feature_values > value]
        
        # Calcular la proporción de cada clase en ambos conjuntos
        left_proportions = np.bincount(left_subset) / len(left_subset)
        right_proportions = np.bincount(right_subset) / len(right_subset)
        
        # Calcular el índice de Gini para este umbral y característica
        gini_left = 1 - sum(left_proportions**2)
        gini_right = 1 - sum(right_proportions**2)
        
        # Calcular el índice ponderado por el tamaño de los conjuntos
        weighted_gini = (len(left_subset) * gini_left + len(right_subset) * gini_right) / len(target_values)
        
        # Actualizar el índice de Gini mínimo
        if weighted_gini < gini_index:
            gini_index = weighted_gini
    
    return gini_index

# Calcular el índice de Gini para la característica 'Edad'
gini_edad = calculate_gini_index(df['Edad'].values, df['Juega al Fútbol'].values)
print(f'Índice de Gini para "Edad": {gini_edad}')

# Calcular el índice de Gini para la característica 'Altura'
gini_altura = calculate_gini_index(df['Altura'].values, df['Juega al Fútbol'].values)
print(f'Índice de Gini para "Altura": {gini_altura}')


Índice de Gini para "Edad": 0.4
Índice de Gini para "Altura": 0.3
